It's my first public kernel.😂  I tried some easy methods and graphs for EDA.

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../input/top-play-store-games/android-games.csv')

In [ ]:
df.shape

In [ ]:
df.head()

We can see the type of 'installs' is character. I want to convert it into numeric.

In [ ]:
df['installs'].value_counts()

In [ ]:
convert_installs = {
    '100.0 k' : 100000,
    '500.0 k' : 500000,
    '1.0 M' : 1000000,
    '5.0 M' : 5000000,
    '10.0 M' :10000000,
    '50.0 M' : 50000000,
    '100.0 M': 100000000,
    '500.0 M': 500000000,
    '1000.0 M': 1000000000,
}
df['installs'] = df['installs'].map(convert_installs)

In [ ]:
df.describe()

In [ ]:
df[df['price'] != 0]   # which means 'paid' == True

There are only 7 charged games.


I want to see whether ratings have relation to the rank. But the number of ratings isn't helpful because the more total ratings, the more 5 star ratings and also 1 star ratings.  

So I will make 'proportion columns' for each ratings, which calculate the proportion of each ratings.

In [ ]:
df['props of 5 star'] = df['5 star ratings'] / df['total ratings']
df['props of 4 star'] = df['4 star ratings'] / df['total ratings']
df['props of 3 star'] = df['3 star ratings'] / df['total ratings']
df['props of 2 star'] = df['2 star ratings'] / df['total ratings']
df['props of 1 star'] = df['1 star ratings'] / df['total ratings']

In [ ]:
df.describe()

Proportions are always between 0 and 1. But maximum value of 'props of 5 star' is 2.220635. And I don't need the original ratings columns. So I dropped those wrong rows and unnecessary columns.

In [ ]:
df = df[df['props of 5 star'] <= 1]
df.drop(['5 star ratings', '4 star ratings', '3 star ratings', '2 star ratings', '1 star ratings'], axis=1, inplace=True)

## Correlation Matrix

Let's check the correlation matrix.

In [ ]:
corr = df.corr()
plt.figure(figsize=(12, 12))
sns.heatmap(corr, square=True, annot=True, linewidths=1.0, cmap='RdBu')

The correlations between 'rank and total ratings', 'rank and installs' are negative numbers because **low values of rank** (ex. 1, 2, 3) means **higher rank** (Top ranks).

Of course 'average rating' is highly related to '5 ~ 1 star ratings'. But interestingly, 'props of 4 star' has almost no (and actually negative) correlation.

And 'average rating' doesn't matter to 'rank'. It's also interesting.

## Distribution of Ratings

In [ ]:
plt.subplots(figsize=(15,7))
sns.kdeplot(df['props of 5 star'], label='5 stars')
sns.kdeplot(df['props of 4 star'], label='4 stars')
sns.kdeplot(df['props of 3 star'], label='3 stars')
sns.kdeplot(df['props of 2 star'], label='2 stars')
sns.kdeplot(df['props of 1 star'], label='1 stars')
plt.legend()

People usually don't give 2 or 3 star ratings.  

Maybe becaues the datas are from the 'TopGames', the value of 'props of 5 star' is much higher than that of the others. Interstingly, some games have less than 50% of 5 star ratings! So I tried to find the rows that 'average rating is less than 3.5'.

In [ ]:
df[df['average rating'] < 3.5]

13 games have 'average rating' under 3.5. Most of them have low rank, but there is also 5th, '모두의 마블'. 

## Ratings by Category

I'll draw boxplots of 'average rating' for each category.

In [ ]:
med_idx = df.groupby('category')['average rating'].median().sort_values(ascending=False).index
df.groupby('category')['average rating'].median().sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(10, 8))
sns.boxplot(x='average rating', y='category', data=df, order=med_idx)

The 'GAME WORD' has the highest median of 'average rating', and that of 'GAME MUSIC' is the lowest.

The 'GAME TRIVIA' has many outliers.

The 'GAME RACING' has really short box, which means low variance. People seem to have given similar ratings. 

Only the 'GAME EDUCATIONAL' has outliers on the higher side. They are almost '5'.

Let's see which category has many ratings. 

In [ ]:
mean_idx = df.groupby('category')['total ratings'].mean().sort_values(ascending=False).index
df.groupby('category')['total ratings'].mean().sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='total ratings', y='category', data=df.groupby('category')['total ratings'].mean().reset_index(), order=mean_idx)